In [1]:
import pandas as pd
import pywikibot
import requests
from tqdm import tqdm

In [2]:
wikipedia = pywikibot.Site('en', 'wikipedia')

# Retrieve the page title from the wiki page id
movie_wiki_id= 27611999
api_url = url = f'https://en.wikipedia.org/w/api.php?action=query&pageids={movie_wiki_id}&format=json'
json_data = requests.get(api_url).json()
movie_title = json_data['query']['pages'][str(movie_wiki_id)]['title']

# Retrieve the movie country from the wiki page
country_property = 'P495'
movie_wikipage = pywikibot.Page(wikipedia, movie_title)
movie_itempage = pywikibot.ItemPage.fromPage(movie_wikipage)
movie_itempage.get()
movie_country = movie_itempage.claims[country_property][0].getTarget().labels['en']
print(f"The movie '{movie_title}' is from '{movie_country}'")

The movie 'Special Effects (film)' is from 'United States of America'


In [3]:
# Load the data
DATA_PATH = 'clean_data/'
movies_df = pd.read_csv(DATA_PATH + 'movies.csv')

# For each movie, retrieve the country from the wiki page and add it to the dataframe
movies_df['country'] = ''
for index, row in tqdm(movies_df.iterrows(), total=movies_df.shape[0]):
	# Retrieve the page title from the wiki page id
	movie_wiki_id = row['wiki_id']
	api_url = url = f'https://en.wikipedia.org/w/api.php?action=query&pageids={movie_wiki_id}&format=json'
	json_data = requests.get(api_url).json()
	try:
		movie_title = json_data['query']['pages'][str(movie_wiki_id)]['title']
	except:
		print(f"Error: movie with wiki_id {movie_wiki_id} not found")
		continue

	# Retrieve the movie country from the wiki page
	movie_wikipage = pywikibot.Page(wikipedia, movie_title)
	movie_itempage = pywikibot.ItemPage.fromPage(movie_wikipage)
	try:
		movie_itempage.get()
		movie_country = movie_itempage.claims[country_property][0].getTarget().labels['en']
		movies_df.loc[index, 'country'] = movie_country
	except:
		print(f"Error: country for movie '{movie_title}' not found")
		continue

	break

# Save the dataframe
movies_df.to_csv('movies.csv', index=False)

  0%|          | 0/81741 [00:02<?, ?it/s]


In [2]:
import requests

api_key = '80e81791f5a7e6ef65685c64eb3ecd22'
movie_id = 27611999
url = f'https://api.themoviedb.org/3/movie/{movie_id}?api_key={api_key}'

response = requests.get(url)
if response.status_code == 200:
	json_data = response.json()
	print(json_data['title'])